In [5]:
import os
import pandas as pd
import unicodedata

### TODO: signal data와 meta data (발전용량, 준공년도 등) 이름 통일하기

- list_value_keys는 호기 여러 개 존재하는 발전소여서 호기별로 분리해서 저장하기

- exception
    * 안동소내, 삼척사택 데이터는 일별 발전량만 있고 시간별 데이터 없어서 사용 못할듯

#### 1. 수동 mapping table 
- key: signal data에서의 발전소 이름
- value: meta data에서의 발전소 이름

In [6]:
# Creating a mapping dictionary between file names and actual power plant names
solar_name_mapping = {
    '감우리': '감우리',
    '남제주소내': '남제주소내태양광',
    '무릉리': '무릉리',
    '부산복합자재창고': '부산자재창고태양광',
    '부산본부': ['부산소내태양광#1', '부산소내태양광#2'],
    '부산수처리장': '부산 수처리장 태양광',
    '부산신항': '부산신항태양광',
    '부산역선상주차장': '부산역 선상 주차장',
    '부산운동장': '부산 운동장부지 태양광',
    #'삼척사택': '삼척사택 태양광',
    '삼척소내': ['삼척 소내태양광 #1', '삼척 소내태양광 #2', '삼척 소내태양광 #3', '삼척소내#4'],
    '세화리': '세화리',
    '송당리': '송당리 태양광',
    '신인천 1_2단계 주차장': '신인천 1,2단계 주차장 태양광',
    '신인천 북측부지': '신인천 북측부지 태양광',
    '신인천 주차장': '신인천 주차장 태양광',
    '신인천본관주차장': '신인천 소내주차장',
    '신인천소내': '신인천소내태양광',
    '신인천전망대': '신인천 전망대 태양광',
    '신인천해수구취수구': '신인천해수취수로태양광',
    '신풍리': '신풍리',
    #'안동소내': ['안동 소내태양광 #1', '안동 소내태양광#2'],
    '영월본부': '영월복합태양광',
    '영월철도부지': '영월 철도부지 태양광',
    '와산리': '와산리',
    '용수리': '제주 용수리 태양광',
    '위미2리': '위미2리태양광',
    '이천D(백사면B)': '백사면 B',
    '이천시 백사면A': '백사면 A',
    '익산 다송리': '익산다송리',
    '인천수산정수장': '인천수산정수장태양광',
    '하동공설운동장': '하동공설운동장태양광',
    '하동변전소': '하동변전소태양광',
    '하동보건소': '하동군보건소태양광',
    '하동본부': ['하동화력태양광 #1', '하동화력태양광 #2 (3개소)', '하동화력태양광 #3', '하동화력태양광 #4', '하동화력 태양광 #5', '하동소내#6'],
    '하동정수장': '하동정수장태양광',
    '하동하수처리장': '하동하수처리장태양광',
    '행원소수력': '행원소수력태양광',
    '화촌주민참여형': '화촌 솔라팜'
}
solar_name_mapping = {
    unicodedata.normalize('NFC', str(k).strip()): v 
    for k, v in solar_name_mapping.items()
}
len(solar_name_mapping)

37

In [9]:
# 데이터 저장할 경로 정의
save_path = '../../data/solar_energy'
os.makedirs(save_path,exist_ok=True)

#### 2. 필요한 Meta data만 solar_name_mapping의 key 활용하여 table로 저장하기

In [10]:
# Signal files
path = '../../data/solar_energy/raw_data'
files = [i for i in os.listdir(path) if '태양광' in i]
files = [
    unicodedata.normalize('NFC', name.strip())
    for name in files
]
files = [f for f in files if '중부발전' not in f]
file_names = [name.split('(주)_')[1].split('태양광')[0].strip() for name in files]
file_names = [
    unicodedata.normalize('NFC', name.strip())
    for name in file_names
]
file_mapping_dict = {}
for i in range(len(files)):
    file_mapping_dict[file_names[i]] = files[i]

# Meta data
meta_data = pd.read_csv(os.path.join(path, '한국남부발전(주)_에너지원별 신재생에너지 발전설비 현황_20250312.csv'),encoding='cp949')
meta_data['발전소명'] = meta_data['발전소명'].astype(str).str.strip()
meta_data['발전소명'] = meta_data['발전소명'].apply(lambda x: unicodedata.normalize('NFC', x))


file_names.sort()
print("file_names:",len(file_names))
print(file_names[:4],'..')

file_names: 39
['감우리', '남제주소내', '무릉리', '부산복합자재창고'] ..


In [11]:
print('하나의 signal file에 meta data는 여러 개 존재하는 경우 (발전호기가 2개 이상인 경우)')
list_value_keys = [k for k, v in solar_name_mapping.items() if isinstance(v, list)]
list_value_keys

하나의 signal file에 meta data는 여러 개 존재하는 경우 (발전호기가 2개 이상인 경우)


['부산본부', '삼척소내', '하동본부']

In [12]:
all_meta_data = []

# 1. 1:1 매핑 가능한 데이터들에 대해서 우선 저장
keys = [i for i in solar_name_mapping.keys() if i not in list_value_keys]
print('1. 1:1 매핑 가능한 데이터들:',len(keys),'\n')

for key in keys:
    meta = meta_data[meta_data['발전소명']==solar_name_mapping[key]].copy()
    meta['name'] = key
    all_meta_data.append(meta[['name','발전소명','구분','용량(MW)','준공년도','비 고']] )

import re
# 2. 발전호기 2개 이상인 경우를 처리
print('2. 발전호기 2개 이상인 경우:',len(list_value_keys))
print('-> name_number 형식으로 name 저장함')
for key in list_value_keys:
    meta = meta_data[meta_data['발전소명'].isin(solar_name_mapping[key])].copy()
    # 발전소명 안의 숫자 추출 & name 생성
    names = []
    for plant_name in meta['발전소명']:
        match = re.search(r'#\s*(\d+)', plant_name)  # '# 숫자' 패턴 찾기
        if match:
            num = match.group(1)
            new_name = f"{key}_{num}"
        else:
            new_name = key  # 숫자가 없으면 그냥 key 사용
        names.append(new_name)

    meta['name'] = names

    all_meta_data.append(meta[['name', '발전소명', '구분', '용량(MW)', '준공년도', '비 고']])
all_meta_data = pd.concat(all_meta_data,ignore_index=True)


1. 1:1 매핑 가능한 데이터들: 34 

2. 발전호기 2개 이상인 경우: 3
-> name_number 형식으로 name 저장함


In [13]:
# 미리 지역 정보를 매핑한 데이터
data = [
    {"광역시": "경기도", "지역": "이천", "발전소": ["이천시 백사면A", "이천D(백사면B)"], "location_name": "이천"},
    {"광역시": "인천광역시", "지역": "인천", "발전소": ["신인천 1_2단계 주차장", "신인천 북측부지", "신인천 주차장", "신인천본관주차장", "신인천소내", "신인천전망대", "신인천해수구취수구", "인천수산정수장"], "location_name": "인천"},
    {"광역시": "강원도", "지역": "삼척", "발전소": ["삼척사택", "삼척소내_1", "삼척소내_2", "삼척소내_3", "삼척소내_4"], "location_name": "동해"},
    {"광역시": "강원도", "지역": "영월", "발전소": ["영월본부", "영월철도부지"], "location_name": "영월"},
    {"광역시": "충청북도", "지역": "음성", "발전소": ["감우리"], "location_name": "충주"},
    {"광역시": "전라북도", "지역": "익산", "발전소": ["익산 다송리"], "location_name": "전주"},
    {"광역시": "경상북도", "지역": "안동", "발전소": ["안동소내"], "location_name": "안동"},
    {"광역시": "경상남도", "지역": "하동", "발전소": ["하동공설운동장", "하동변전소", "하동보건소", "하동본부_1", "하동본부_2", "하동본부_3", "하동본부_4", "하동본부_5", "하동본부_6", "하동정수장", "하동하수처리장", "화촌주민참여형"], "location_name": "진주"},
    {"광역시": "부산", "지역": "부산", "발전소": ["부산복합자재창고", "부산본부_1", "부산본부_2", "부산수처리장", "부산신항", "부산역선상주차장", "부산운동장"], "location_name": "부산"},
    {"광역시": "제주특별자치도", "지역": "제주", "발전소": ["송당리", "남제주소내", "제주소내", "행원소수력", "용수리"], "location_name": "제주"},
    {"광역시": "제주특별자치도", "지역": "제주", "발전소": ["세화리", "신풍리", "와산리"], "location_name": "성산일출"},
    {"광역시": "제주특별자치도", "지역": "제주", "발전소": ["위미2리", "무릉리"], "location_name": "서귀포"},
]

df = pd.DataFrame(data)
df

,광역시,지역,발전소,location_name
0,경기도,이천,"[이천시 백사면A, 이천D(백사면B)]",이천
1,인천광역시,인천,"[신인천 1_2단계 주차장, 신인천 북측부지, 신인천 주차장, 신인천본관주차장, 신...",인천
2,강원도,삼척,"[삼척사택, 삼척소내_1, 삼척소내_2, 삼척소내_3, 삼척소내_4]",동해
3,강원도,영월,"[영월본부, 영월철도부지]",영월
4,충청북도,음성,[감우리],충주
5,전라북도,익산,[익산 다송리],전주
6,경상북도,안동,[안동소내],안동
7,경상남도,하동,"[하동공설운동장, 하동변전소, 하동보건소, 하동본부_1, 하동본부_2, 하동본부_3...",진주
8,부산,부산,"[부산복합자재창고, 부산본부_1, 부산본부_2, 부산수처리장, 부산신항, 부산역선상...",부산
9,제주특별자치도,제주,"[송당리, 남제주소내, 제주소내, 행원소수력, 용수리]",제주


In [14]:
# 먼저 발전소 이름과 location_name을 매핑한 dict 생성
mapping = {}
for _, row in df.iterrows():
    for plant in row["발전소"]:
        mapping[plant.strip()] = row["location_name"]

# all_meta_data의 name과 mapping 키를 비교해서 location_name 할당
all_meta_data["location_name"] = all_meta_data["name"].map(mapping)

all_meta_data = pd.merge(all_meta_data,df[['location_name','광역시']],on='location_name',how='left')#.drop_duplicates(subset=['location_name']).to_csv(os.path.join(data_path, 'location_mapping.csv'), index=False)

all_meta_data

,name,발전소명,구분,용량(MW),준공년도,비 고,location_name,광역시
0,감우리,감우리,자체,0.5540,2021,RPS,충주,충청북도
1,남제주소내,남제주소내태양광,자체,0.1960,2012,RPS,제주,제주특별자치도
2,무릉리,무릉리,자체,0.8090,2021,RPS,서귀포,제주특별자치도
3,부산복합자재창고,부산자재창고태양광,자체,0.1870,2012,RPS,부산,부산
4,부산수처리장,부산 수처리장 태양광,자체,0.1110,2017,RPS,부산,부산
5,부산신항,부산신항태양광,자체,0.1150,2011,RPA,부산,부산
6,부산역선상주차장,부산역 선상 주차장,자체,0.8344,2020,RPS,부산,부산
7,부산운동장,부산 운동장부지 태양광,자체,1.4000,2017,RPS,부산,부산
8,세화리,세화리,자체,0.9990,2021,RPS,성산일출,제주특별자치도
9,송당리,송당리 태양광,자체,0.9970,2019,RPS,제주,제주특별자치도


In [15]:
all_meta_data.to_csv(os.path.join(save_path,'meta_data.csv'),index=False)
all_meta_data

,name,발전소명,구분,용량(MW),준공년도,비 고,location_name,광역시
0,감우리,감우리,자체,0.5540,2021,RPS,충주,충청북도
1,남제주소내,남제주소내태양광,자체,0.1960,2012,RPS,제주,제주특별자치도
2,무릉리,무릉리,자체,0.8090,2021,RPS,서귀포,제주특별자치도
3,부산복합자재창고,부산자재창고태양광,자체,0.1870,2012,RPS,부산,부산
4,부산수처리장,부산 수처리장 태양광,자체,0.1110,2017,RPS,부산,부산
5,부산신항,부산신항태양광,자체,0.1150,2011,RPA,부산,부산
6,부산역선상주차장,부산역 선상 주차장,자체,0.8344,2020,RPS,부산,부산
7,부산운동장,부산 운동장부지 태양광,자체,1.4000,2017,RPS,부산,부산
8,세화리,세화리,자체,0.9990,2021,RPS,성산일출,제주특별자치도
9,송당리,송당리 태양광,자체,0.9970,2019,RPS,제주,제주특별자치도


In [18]:
pd.DataFrame(all_meta_data['광역시'].value_counts())



,광역시
경상남도,12
제주특별자치도,9
인천광역시,8
부산,7
강원도,6
경기도,2
충청북도,1
전라북도,1


In [19]:
pd.DataFrame(all_meta_data['location_name'].value_counts())



,location_name
진주,12
인천,8
부산,7
제주,4
동해,4
성산일출,3
서귀포,2
영월,2
이천,2
충주,1


In [20]:
all_meta_data['location_name'].unique()


array(['충주', '제주', '서귀포', '부산', '성산일출', '인천', '영월', '이천', '전주', '진주',
       '동해'], dtype=object)